<a href="https://colab.research.google.com/github/yaretdelanuez/TFG-YARET/blob/main/tfg_redes_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Montamos el google drive para acceder al csv de variables_latentes**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Comprobamos el directorio y a continuación ponemos al que queremos acceder del google drive**

In [2]:
!pwd
import os
os.chdir('/content/drive/My Drive/TFG')
!pwd

/content
/content/drive/My Drive/TFG


In [5]:
#Importamos librerias que vamos a usar para la ANN
import numpy as np
import pandas as pd
import tensorflow as tf
import random
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import sklearn.metrics as metrics
from sklearn.model_selection import KFold

In [6]:
#Para que sea determinista, la semilla
tf.keras.utils.set_random_seed(0)
print(tf.random.uniform([1]))

tf.Tensor([0.29197514], shape=(1,), dtype=float32)


***Cargamos los datos desde el fichero que exportamos de PLS-SEM***


In [7]:
datos = pd.read_csv('variables_latentes.csv' ,sep=',', index_col=0)
datos.head()
#Separamos el conjutno de datos de la x e y
datos_x = datos.loc[:, ["orgSupport", "jobSatisfaction"]]
datos_y = datos.loc[:, ["performance"]]

#print(datos_x)
#print(datos_y)


**Preparamos las variables para los datos de entrenamiento 75% y test 25%**

In [8]:
x_entrenamiento, x_test, y_entrenamiento, y_test = train_test_split( datos_x, datos_y, test_size=0.25)
#print(x_entrenamiento,x_test,y_entrenamiento,y_test)

**En esta parte jugaremos con las capas de nuestra red para analizar con cual daremos mejores resultados**

In [9]:
#Inicializar la ann
ann_model = tf.keras.models.Sequential()
#Input layer
ann_model.add(tf.keras.layers.Dense(units=2, activation='relu', input_shape=(2,)))
#Multiple hidden layers
ann_model.add(tf.keras.layers.Dense(units=8, activation='relu'))
ann_model.add(tf.keras.layers.Dense(units=24, activation='relu'))
#ann_model.add(tf.keras.layers.Dense(units=8, activation='relu'))
#ann_model.add(tf.keras.layers.Dense(units=2, activation='relu'))
#Output layer
ann_model.add(tf.keras.layers.Dense(units=1, activation='linear'))

#Adam es un método de descenso de gradiente estocástico que se basa en la estimación adaptativa de momentos de primer y segundo orden.
ann_model.compile(optimizer = 'adam', loss = 'mse', metrics = ['mse'])



In [10]:
#Ver la configuración del modelo
ann_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2)                 6         
                                                                 
 dense_1 (Dense)             (None, 8)                 24        
                                                                 
 dense_2 (Dense)             (None, 24)                216       
                                                                 
 dense_3 (Dense)             (None, 1)                 25        
                                                                 
Total params: 271
Trainable params: 271
Non-trainable params: 0
_________________________________________________________________


**Realizamos el entrenamiento, la predicción y sacamos los valores de rmse, r2, para la configuración básica**

In [11]:
#Inicializamos

#Entrenamiento
ann_model.fit(x_entrenamiento, y_entrenamiento, epochs=150, batch_size=10, verbose = 0) #verbose quita la ejecución la vista  verbose = 0
#Predicción
y_prediction = ann_model.predict(x_test)

print("--------------------------------")
#print(y_prediction)
#MSE
mse = mean_squared_error(y_test, y_prediction)
print("MSE:" ,mse)
#RMSE
rmse = math.sqrt(mean_squared_error(y_test, y_prediction))
print("RMSE: ",rmse)
#RMSEA
rmsea = math.sqrt(mean_squared_error(y_test, y_prediction) / ((len(datos)*len(y_prediction))-1))
print("RMSEA:" ,rmsea)
#R2
r2 =r2_score(y_test,y_prediction)
print("r2: ", r2)

print(mse, rmse, rmsea, r2)

2/2 [==============================] - 0s 7ms/step
--------------------------------
MSE: 1.006460149373985
RMSE:  1.0032248747783246
RMSEA: 0.010057929251871897
r2:  0.12488647351886228
1.006460149373985 1.0032248747783246 0.010057929251871897 0.12488647351886228


**Red con cross validation**

In [12]:
#cross validation

def train_test(batch_size, epochs):

    #Por iteracción contiene el mse
    mse_iter = []
    #Por iteracción contiene el rmsea
    rmsea_iter = []
    #Por iteracción contiene el rmse
    rmse_iter = []
    #Por iteracción contiene el r2
    r2_iter = []

    #variable para numero de veces que dividimos la muestra
    kf=10

    #kfold para cross validation
    kfold = KFold(n_splits=kf, shuffle=False)

    for train_index, test_index in kfold.split(datos):
        X_train = datos.iloc[train_index].loc[:, ["jobSatisfaction", "orgSupport"]]
        y_train = datos.iloc[train_index].loc[:, ["performance"]]
        X_test = datos.iloc[test_index].loc[:, ["jobSatisfaction", "orgSupport"]]
        y_test = datos.iloc[test_index].loc[:, ["performance"]]
        #entrenamiento
        ann_model.fit(X_train, y_train, batch_size = batch_size, epochs = epochs)
        #predecir
        y_prediction = ann_model.predict(X_test)

        #print("predichos" , len(y_prediction))
        #print(y_prediction)

        #Cálculo mse por iteración
        mse_iter.append(mean_squared_error(y_test, y_prediction))
        #RMSE
        rmse_iter.append(math.sqrt(mean_squared_error(y_test, y_prediction)))
        #RMSEA = sqrt(MSE / ((n * p) - 1))
        #n tamaño muestra
        #p número de parametros estimados en la muestra
        #Cálculo rmsea por iteración
        rmsea_iter.append(math.sqrt(mean_squared_error(y_test, y_prediction) / ((len(datos)*len(y_prediction))-1)))

        #Cálculo r2 por iteración
        r2_iter.append(r2_score(y_test,y_prediction))
    #mse división por numero de iteraciones
    mse = 0
    #print(mse_iter)
    for i in mse_iter:
      mse += i
    mse = mse/kf
    print("mse: ", mse)
    #rmse división por numero de iteraciones
    rmse = 0
    print("Iteraciones rmse: ",rmse_iter)
    for i in rmse_iter:
      rmse += i
    rmse = rmse/kf
    print("rmse: ", rmse)
    #rmsea división por numero de iteraciones
    #print(rmsea_iter)
    rmsea = 0
    for i in rmsea_iter:
          rmsea += i
    rmsea = rmsea/kf
    print("rmsea: ",rmsea)
    #r2 división por numero de iteraciones
    print("Iteraciones r2: ",r2_iter)
    r2 = 0
    for i in r2_iter:
          r2 += i
    r2 = r2/kf
    print("r2: ", r2)


    print(mse, rmse, rmsea, r2)


In [13]:
#Realizar el test de cross validation
train_test(20,150)


Epoch 1/150
9/9 [==============================] - 0s 2ms/step - loss: 1.1227 - mse: 1.1227
Epoch 2/150
9/9 [==============================] - 0s 2ms/step - loss: 1.1003 - mse: 1.1003
Epoch 3/150
9/9 [==============================] - 0s 2ms/step - loss: 1.0770 - mse: 1.0770
Epoch 4/150
9/9 [==============================] - 0s 2ms/step - loss: 1.0584 - mse: 1.0584
Epoch 5/150
9/9 [==============================] - 0s 2ms/step - loss: 1.0426 - mse: 1.0426
Epoch 6/150
9/9 [==============================] - 0s 2ms/step - loss: 1.0298 - mse: 1.0298
Epoch 7/150
9/9 [==============================] - 0s 2ms/step - loss: 1.0160 - mse: 1.0160
Epoch 8/150
9/9 [==============================] - 0s 2ms/step - loss: 1.0056 - mse: 1.0056
Epoch 9/150
9/9 [==============================] - 0s 2ms/step - loss: 0.9973 - mse: 0.9973
Epoch 10/150
9/9 [==============================] - 0s 1ms/step - loss: 0.9939 - mse: 0.9939
Epoch 11/150
9/9 [==============================] - 0s 2ms/step - loss: 0.9895 


## **Sklearn con RandomForestRegressor**

In [14]:
#Importamos librerias necesarias
from sklearn.ensemble import RandomForestRegressor
from sklearn import set_config


**Establecer ramdom Forest Regression**

In [15]:
#configuración por defecto
set_config(print_changed_only = False)
rfr = RandomForestRegressor()


In [16]:
#Por iteracción contiene el mse
mse_iter = []
#Por iteracción contiene el rmsea
rmsea_iter = []
#Por iteracción contiene el rmse
rmse_iter = []
#Por iteracción contiene el r2
r2_iter = []

#variable para numero de veces que dividimos la muestra
kf=10

#kfold para cross validation
kfold = KFold(n_splits=kf, shuffle=False)
for train_index, test_index in kfold.split(datos):
    X_train = datos.iloc[train_index].loc[:, ["jobSatisfaction", "orgSupport"]]
    y_train = datos.iloc[train_index].loc[:, ["performance"]]
    X_test = datos.iloc[test_index].loc[:, ["jobSatisfaction", "orgSupport"]]
    y_test = datos.iloc[test_index].loc[:, ["performance"]]
    #entrenar
    rfr.fit(X_train, y_train.values.ravel())
    #predecir
    y_prediction = rfr.predict(X_test)
    #print(y_prediction)

    #Cálculo mse por iteración
    mse_iter.append(mean_squared_error(y_test, y_prediction))
    #RMSE
    rmse_iter.append(math.sqrt(mean_squared_error(y_test, y_prediction)))
    #RMSEA = sqrt(MSE / ((n * p) - 1))
    #n tamaño muestra
    #p número de parametros estimados en la muestra
    #Cálculo rmsea por iteración
    rmsea_iter.append(math.sqrt(mean_squared_error(y_test, y_prediction) / ((len(datos)*len(y_prediction))-1)))
    #Cálculo r2 por iteración
    r2_iter.append(r2_score(y_test,y_prediction))


#mse división por numero de iteraciones
mse = 0
#print(mse_iter)
for i in mse_iter:
    mse += i
    mse = mse/kf
print("mse: ", mse)
#rmse división por numero de iteraciones
rmse = 0
print("Iteraciones rmse: ",rmse_iter)
for i in rmse_iter:
    rmse += i
    rmse = rmse/kf
print("rmse: ", rmse)
#rmsea división por numero de iteraciones
#print(rmsea_iter)
rmsea = 0
for i in rmsea_iter:
    rmsea += i
    rmsea = rmsea/kf
print("rmsea: ",rmsea)
#r2 división por numero de iteraciones
print("Iteraciones r2: ",r2_iter)
r2 = 0
for i in r2_iter:
    r2 += i
    r2 = r2/kf
print("r2: ", r2)

print(mse, rmse, rmsea, r2)

mse:  0.12043111796805335
Iteraciones rmse:  [1.021905331086519, 1.0574127837538827, 1.11333316458062, 0.9294631012642837, 1.1888873488531282, 0.9163480693585029, 1.0066871399431594, 1.082689398397338, 0.8861678121035687, 1.0549631056297661]
rmse:  0.11555180440369148
rmsea:  0.0018753079399983733
Iteraciones r2:  [-0.24176213793195056, -0.47914625312703407, 0.10550586947665797, -0.1780707974754474, -0.24127093246465758, 0.06150709454352832, 0.039755503159414673, 0.03335985664362917, -0.013142443900374667, -0.23496602908128272]
r2:  -0.023590335395503
0.12043111796805335 0.11555180440369148 0.0018753079399983733 -0.023590335395503


----------------------------------------------------------